# FARA
> Play FARA

- https://playfara.com

Motivation: A large part of playing FARA involves going through certain routines. These routines are fun to figure out, but their novelty wears off as the game progresses. When starting a new game of FARA, it is preferable to thoroughly search, harvest, and craft in the initial locality. It would be nice to automate away this early game routine. Another such routine is preparing for specific journeys which could involve harvesting and preparing shelter, meals, and rest.

- [ ] **Available Moves**: Learn what the available moves are by parsing the gamelog. And maybe in which situations they are useful.
- [ ] **Satchel**: Learn to craft a burlap satchel in the first area.


## Available Moves

- Given a game log ending with `> `, output the next command.
- Start with a game log.

## Interface via Selenium

In [26]:
from selenium.webdriver.common.keys import Keys
import random
import re
import io
import sys

lm = [
    Keys.ARROW_DOWN, Keys.ARROW_LEFT, Keys.ARROW_UP, Keys.ARROW_RIGHT,
    '.', 'x'
]

#################
# Random Strategy
# effectively, causes staff to be used less often
# longer games, but not necessarily more points.
costs = [1/cost for cost in [1,1,1,1,1,10]]
def random_move(b):
    return random.choices(lm, costs)
#################

#################
# Manual Strategy [WIP]
def human_move(b):
    return sys.stdin.read(1)
#################

def board(g):
    # .text is a little too magical (writes \n instead of <br>, etc..)
    return g.find_element(by="id", value="tsv").get_attribute('innerHTML')

def game_score(g):
    return None
#     # .text is easier to regex than innerHTML
#     game_over = re.search("Your final score is (\d+) after ", g.text)
#     # scoring function is the score, but alternative scoring
#     # functions could count kills, visits, keys, etc..
#     return game_over.group(1) if game_over else None

def turn(g):
    s = game_score(g)
    b = board(g)
    # How we get m will by the strategy
    m = random_move(b)
    # m = human_move(board(g))
    if not s: g.send_keys(m)
    return (s, f"{b}\t{m}")

def turn_score(b, g):
    return 0
#     try:
#         return int(re.match("Self: (?:\<[^>]+\>)*(\w+)", b.split("\t")[2500]).group(1))
#     except ValueError:
#         return int(re.search("Your final score is (\d+) after ", g.text).group(1))

def reward(b1, b2, g):
    return turn_score(b2, g) - turn_score(b1, g)

# def game(driver, save_as=None):
#     driver.get("https://brianiscreative.itch.io/fara")
#     g = driver.find_element(by="tag name", value="body")
#     [g.send_keys(c) for c in ["/stopsharing", "/hd", "/showinput", "/logging"]]
#     score = None
#     history = []
#     while score is None:
#         score, b = turn(g)
#         r = reward(b, board(g), g)
#         # make sure not to leak the moves and rewards in training
#         history.append(f"{b}\t{r}")
#     # print(history)
#     if save_as:
#         with open(f"{save_as}", "w") as f: f.write("\n".join(history))
#     return int(score), len(history)

## Starting FARA 

Initialize the Selenium driver.
Unless you use headless mode, you'll want to keep this new Firefox window open as long as you are playing.

At this point, we will go through the steps necessary to start a game. And do so in a way that the game log can be saved. We'll use the log later to generate training data.

Time to get it saving somewhere more manageable. I'll try to overwrite the same file for now though.

In [198]:
! rm -f /tmp/fara/game/*
! ls /tmp/fara/game/


In [199]:
import os

from selenium import webdriver

# TODO: patch to a class
def set_logging_dir(p):
  log_dir = p
  ! mkdir -p $p
  ! rm $p/*
  fp = webdriver.FirefoxProfile()

  fp.set_preference("browser.download.folderList",2)
  fp.set_preference("browser.download.manager.showWhenStarting",False)
  fp.set_preference("browser.download.dir", p)
  # FIXME: these aren't working.
  # fp.set_preference("browser.helperApps.neverAsk.saveToDisk", "text/plain")
  fp.set_preference("browser.helperApps.alwaysAsk.force", False)

  return (fp, p)

Okay. Now that we're moving along, it's a good time to start getting organized about how we start the game.

In [202]:
from selenium.webdriver.firefox.options import Options
import time

def start_game():
  driver = webdriver.Firefox()
  try: driver.quit()
  except: pass
  options = Options()
  # options.headless = True
  fp, log_dir = set_logging_dir("/tmp/fara/game")
  driver = webdriver.Firefox(options=options,
    firefox_profile=fp
  )
  time.sleep(1)
  driver.get("https://brianiscreative.itch.io/fara")
  time.sleep(1)
  # TODO: Wait until it's loaded
  from selenium.webdriver.common.by import By
  driver.find_element_by_class_name("load_iframe_btn").click()
  time.sleep(1)
  # TODO: Wait? until it's loaded
  g = driver.find_element(by="tag name", value="body")
  [g.send_keys(c+"\n") for c in ["/setname bot", "/setclass soldier"]]
  time.sleep(1)
  [g.send_keys(c+"\n") for c in ["/1", "/setdesc", "/stopsharing", "/hd", "/showinput", "/logging"]]
  
  # Since the neverAsk option isn't working, we'll need to force the save via the `/savelog` command.
  # The user will need to click "Save File" and "Do this automatically for files like this from now on."
  g.send_keys("/savelog\n")
  
  return g, log_dir


In [203]:
%%time
g, log_dir = start_game()

rm: /tmp/fara/game/*: No such file or directory
CPU times: user 35.2 ms, sys: 25.4 ms, total: 60.6 ms
Wall time: 14 s


The log is now ready to be saved. 

If it was set up by the expectations above, the log should be output to a file that should starts with `The Life and Times of bot the Soldier` and ends with `.txt`.

For now the gamelog is cumulative, but it will be possible [to clear it at some point](https://discord.com/channels/448497182392451073/448498300723789845/814283733988933652). 

Also at some point, we can try to autosave the file with this FAQ: https://selenium-python.readthedocs.io/faq.html?highlight=file#how-to-auto-save-files-using-custom-firefox-profile

## Making Moves

Let's set up a short function to make a move and output the game log after each move.

In [154]:
def move(m):
  [g.send_keys(x + "\n") for x in [m, "/savelog"]]
  ! mv '{log_dir + "/The Life and Times of bot the Soldier.txt"}' '{log_dir + "/log"}'
  ! ls -latr {log_dir}
  

In [155]:
! echo $log_dir

/tmp/fara/game


In [160]:
move("/look")

total 24
drwxr-xr-x  3 ewolfson  wheel    96 Feb 28 18:28 ..
-rw-r--r--@ 1 ewolfson  wheel   664 Feb 28 18:56 The Life and Times of bot the Soldier(1).txt
-rw-r--r--@ 1 ewolfson  wheel  4306 Feb 28 18:57 log
drwxr-xr-x  4 ewolfson  wheel   128 Feb 28 18:57 .


There are a lot of moves that can be made.
We are interested in training the bot to work in exploration mode. Not the travel mode.

Let's list a few available moves.

In [180]:

MOVES = ["/" + item for sublist in
          [
            ["move " + d for ds in ["n ne e se s sw w nw".split(" ")] for d in ds],
            ["look", "use", "recipes"]
          ] for item in sublist
        ]
MOVES

['/move n',
 '/move ne',
 '/move e',
 '/move se',
 '/move s',
 '/move sw',
 '/move w',
 '/move nw',
 '/look',
 '/use',
 '/recipes']

In [184]:
[item for sublist in [
  ["move " + x for xs in ["n ne e se s sw w nw".split(" ")] for x in xs],
  ["look", "harvest"],
  ["craft " + x for x in ["burlap satchel"]]
] for item in sublist]

['move n',
 'move ne',
 'move e',
 'move se',
 'move s',
 'move sw',
 'move w',
 'move nw',
 'look',
 'harvest',
 'craft burlap satchel']

Interesting. So, we do need some understanding of English to go from `/craft burlap satchel` to `/craft roll of burlap`.

In [172]:
["move " + item for sublist in ["n ne e se s sw w nw".split(" ")] for item in sublist]

TypeError: can only concatenate str (not "list") to str